In [2]:
%pip install google-generativeai langchain langchain_google_genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()

True

In [8]:
import os
import json
import re
import google.generativeai as genai

# Step 1: Setup Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [9]:
problem_statement = """Metro station wants to establish a TicketDistributor machine that issues tickets for
passengers travelling on metro rails. Travellers have options of selecting a ticket for a single
trip, round trips or multiple trips. They can also issue a metro pass for regular passengers or
a time card for a day, a week or a month according to their requirements. The discounts on
tickets will be provided to frequent travelling passengers. The machine is also supposed to
read the metro pass and time cards issued by the metro counters or machine. The ticket rates
differ based on whether the traveller is a child or an adult. The machine is also required to
recognize original as well as fake currency notes. The typical transaction consists of a user
using the display interface to select the type and quantity of tickets and then choosing a
payment method of either cash, credit/debit card or smartcard. The tickets are printed and
dispensed to the user. Also, the messaging facilities after every transaction are required on
the registered number. The system can also be operated comfortably by a touch-screen. A
large number of heavy components are to be used. We do not want our system to slow down,
and also the usability of the machine.
The TicketDistributor must be able to handle several exceptions, such as aborting the
transaction for incomplete transactions, the insufficient amount given by the travellers to the
machine, money return in case of an aborted transaction, change return after a successful
transaction, showing insufficient balance in the card, updated information printed on the
tickets e.g. departure time, date, time, price, valid from, valid till, validity duration, ticket
issued from and destination station. In case of exceptions, an error message is to be displayed.
We do not want user feedback after every development stage but after every two stages to
save time. The machine is required to work in a heavy load environment such that in the
morning and evening time on weekdays, and weekends performance and efficiency would
not be affected."""

### Identify Classes

In [10]:
def identify_classes(problem_statement):
    """
    Identifies potential class names from a given problem statement using Gemini API with CoT prompting.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to extract potential class names from a given problem statement. 
    Follow this step-by-step approach:

    1. Identify the key **nouns** representing domain concepts.
    2. If a noun represents a role or system entity, classify it as a **class**.
    3. Ignore verbs and adjectives.
    4. Output the result in a structured JSON format like this:
       {{"classes": ["Class1", "Class2", "Class3"]}}

    Problem Statement:
    {problem_statement}

    Now extract the class names based on the above rules.
    """

    model = genai.GenerativeModel("gemini-1.5-flash")  # Using Gemini Pro model
    response = model.generate_content(prompt, generation_config={"temperature": 0, "top_p": 1, "top_k": 1})
    
    # Debugging: Print raw response
    response_text = response.text.strip()
    print("Raw Response from Gemini:\n", response_text)

    # Remove triple backticks and 'json' keyword
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))  # Use repr() to detect hidden characters

     # Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("classes", {})  
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

identified_classes = identify_classes(problem_statement)
print(identified_classes)


Raw Response from Gemini:
 ```json
{
  "classes": ["TicketDistributor", "Passenger", "Ticket", "MetroPass", "TimeCard", "Transaction", "DisplayInterface", "PaymentMethod", "CurrencyNote", "Error", "ErrorMessage", "TouchScreen"]
}
```
Cleaned JSON:
 '{\n  "classes": ["TicketDistributor", "Passenger", "Ticket", "MetroPass", "TimeCard", "Transaction", "DisplayInterface", "PaymentMethod", "CurrencyNote", "Error", "ErrorMessage", "TouchScreen"]\n}'
['TicketDistributor', 'Passenger', 'Ticket', 'MetroPass', 'TimeCard', 'Transaction', 'DisplayInterface', 'PaymentMethod', 'CurrencyNote', 'Error', 'ErrorMessage', 'TouchScreen']


```json
{
  "classes": ["TicketDistributor", "Passenger", "Ticket", "MetroPass", "TimeCard", "Transaction", "DisplayInterface", "Payment", "CashPayment", "CreditCardPayment", "DebitCardPayment", "SmartcardPayment", "CurrencyNote", "Error", "ErrorMessage", "TouchScreen"]
}
```


### Identify Attributes

In [11]:
def identify_attributes(problem_statement, classes):
    """
    Identifies attributes for each class extracted from the problem statement using Gemini API with CoT prompting.
    Ensures deterministic output and structured JSON format.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to extract **attributes** for each identified class from the given problem statement.

    **Rules to Identify Attributes:**
    1. Attributes describe **properties or characteristics** of a class.
    2. Ignore verbs, complex entities, and relationships.
    3. Common attributes include **names, IDs, types, and statuses**.
    4. Output the result strictly in **JSON format**, like this:
       {{
           "attributes": {{
               "Class1": ["attribute1", "attribute2"],
               "Class2": ["attribute1", "attribute2"]
           }}
       }}

    **Classes Identified:**
    {classes}

    **Problem Statement:**
    {problem_statement}

    Now extract the attributes for each class.
    """

    model = genai.GenerativeModel("gemini-1.5-flash") 
    response = model.generate_content(prompt, generation_config={"temperature": 0, "top_p": 1, "top_k": 1})

     # Debugging: Print raw response
    response_text = response.text.strip()
    print("Raw Response from Gemini:\n", response_text)

    # Remove triple backticks and 'json' keyword
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))  # Use repr() to detect hidden characters

    # Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("attributes", {})  # Extract attributes dictionary
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

identified_attributes = identify_attributes(problem_statement, identified_classes)

print("Extracted Attributes:\n", identified_attributes)

Raw Response from Gemini:
 ```json
{
  "attributes": {
    "TicketDistributor": ["ID"],
    "Passenger": ["ID", "type", "isFrequentTraveler"],
    "Ticket": ["ID", "type", "price", "departureTime", "date", "validFrom", "validTill", "validityDuration", "issuedFrom", "destinationStation"],
    "MetroPass": ["ID", "type", "validityDuration"],
    "TimeCard": ["ID", "type", "validityDuration"],
    "Transaction": ["ID", "status", "amount"],
    "DisplayInterface": ["type"],
    "PaymentMethod": ["type"],
    "CurrencyNote": ["ID", "isOriginal", "value"],
    "Error": ["ID", "type", "message"],
    "ErrorMessage": ["message"],
    "TouchScreen": ["size"]
  }
}
```
Cleaned JSON:
 '{\n  "attributes": {\n    "TicketDistributor": ["ID"],\n    "Passenger": ["ID", "type", "isFrequentTraveler"],\n    "Ticket": ["ID", "type", "price", "departureTime", "date", "validFrom", "validTill", "validityDuration", "issuedFrom", "destinationStation"],\n    "MetroPass": ["ID", "type", "validityDuration"],\n   

### Identify Relationships

In [12]:
def identify_relationships(problem_statement, classes, attributes):
    """
    Identifies relationships between extracted classes using Gemini API with CoT prompting.
    Extracts Association, Generalization (Inheritance), Aggregation, and Composition.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **all types of relationships** between the given classes based on the problem statement.

    **Types of Relationships:**
    1️⃣ **Association** - One class interacts with another. Example: "User borrows Book".
    2️⃣ **Generalization (Inheritance)** - One class is a specialized form of another. Example: "Admin is a subclass of User".
    3️⃣ **Aggregation** - One class is made up of another, but they have independent lifecycles. Example: "Library has Books, but Books exist independently".
    4️⃣ **Composition** - A stronger form of Aggregation, where the part **cannot exist** without the whole. Example: "House has Rooms, Rooms cannot exist without a House".

    **Step-by-Step Process:**
    - Identify the **verbs** and **context clues** in the problem statement to find relationships.
    - Classify the relationship into one of the four types above.
    - Provide the relationships in **strict JSON format**, like this:
      ```json
      {{
          "relationships": [
              {{"source": "ClassA", "target": "ClassB", "type": "Generalization", "description": "ClassA is a subclass of ClassB"}},
              {{"source": "ClassC", "target": "ClassD", "type": "Aggregation", "description": "ClassC contains ClassD but ClassD can exist independently"}},
              {{"source": "ClassE", "target": "ClassF", "type": "Composition", "description": "ClassE owns ClassF and ClassF cannot exist without ClassE"}}
          ]
      }}
      ```

    **Classes Identified:**
    {classes}

    **Attributes Identified:**
    {attributes}

    **Problem Statement:**
    {problem_statement}

    Now extract **all types of relationships** between the classes.
    """

    model = genai.GenerativeModel("gemini-1.5-flash") 
    response = model.generate_content(prompt, generation_config={"temperature": 0, "top_p": 1, "top_k": 1})

   # Debugging: Print raw response
    response_text = response.text.strip()

    # Remove triple backticks and 'json' keyword
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))  # Use repr() to detect hidden characters

    # Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("relationships", {})  
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

identified_relationships = identify_relationships(problem_statement, identified_classes, identified_attributes)

print("Relationships Identified:", identified_relationships)

Cleaned JSON:
 '{\n  "relationships": [\n    {\n      "source": "TicketDistributor",\n      "target": "Ticket",\n      "type": "Composition",\n      "description": "The TicketDistributor issues and manages Tickets; Tickets cannot exist independently without the distributor."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "MetroPass",\n      "type": "Association",\n      "description": "The TicketDistributor reads and processes MetroPasses."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "TimeCard",\n      "type": "Association",\n      "description": "The TicketDistributor reads and processes TimeCards."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "Transaction",\n      "type": "Composition",\n      "description": "The TicketDistributor creates and manages Transactions; Transactions are intrinsically tied to the distributor."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "DisplayInterface",\n 

### Generate plantUML Script

In [13]:
import os
import json

def generate_plantuml(classes, attributes, relationships, output_file="class_diagram2.puml"):
    """
    Generates a PlantUML class diagram from identified classes, attributes, and relationships.
    """
    plantuml_code = "@startuml\n\n"

    # Define Classes and Attributes
    for cls in classes:
        plantuml_code += f"class {cls} {{\n"
        if cls in attributes:
            for attr in attributes[cls]:
                plantuml_code += f"  {attr}\n"
        plantuml_code += "}\n\n"

    # Define relationships
    for rel in relationships:
        source = rel["source"]
        target = rel["target"]
        rel_type = rel["type"]
        description = rel["description"]

        if rel_type == "Association":
            plantuml_code += f"{source} --> {target} : {description}\n"
        elif rel_type == "Generalization":
            plantuml_code += f"{source} --|> {target} : {description}\n"
        elif rel_type == "Aggregation":
            plantuml_code += f"{source} o-- {target} : {description}\n"
        elif rel_type == "Composition":
            plantuml_code += f"{source} *-- {target} : {description}\n"

        #  # Extract only the **first verb** from the description
        # behavior = description.split()[0] if description else ""

        # if rel_type == "Association":
        #     plantuml_code += f"{source} --> {target} : {behavior}\n"
        # elif rel_type == "Generalization":
        #     plantuml_code += f"{source} --|> {target} : {behavior}\n"
        # elif rel_type == "Aggregation":
        #     plantuml_code += f"{source} o-- {target} : {behavior}\n"
        # elif rel_type == "Composition":
        #     plantuml_code += f"{source} *-- {target} : {behavior}\n"

    plantuml_code += "\n@enduml"

    # Save to file
    with open(output_file, "w") as file:
        file.write(plantuml_code)

    print(f"PlantUML file '{output_file}' generated successfully.")

# 🛠 Debugging Check
print("Classes Extracted:", identified_classes)
print("Attributes Extracted:", identified_attributes)
print("Relationships Extracted:", identified_relationships)

# Generate UML
generate_plantuml(identified_classes, identified_attributes, identified_relationships)

# To render, use: `plantuml class_diagram.puml` in the terminal


Classes Extracted: ['TicketDistributor', 'Passenger', 'Ticket', 'MetroPass', 'TimeCard', 'Transaction', 'DisplayInterface', 'PaymentMethod', 'CurrencyNote', 'Error', 'ErrorMessage', 'TouchScreen']
Attributes Extracted: {'TicketDistributor': ['ID'], 'Passenger': ['ID', 'type', 'isFrequentTraveler'], 'Ticket': ['ID', 'type', 'price', 'departureTime', 'date', 'validFrom', 'validTill', 'validityDuration', 'issuedFrom', 'destinationStation'], 'MetroPass': ['ID', 'type', 'validityDuration'], 'TimeCard': ['ID', 'type', 'validityDuration'], 'Transaction': ['ID', 'status', 'amount'], 'DisplayInterface': ['type'], 'PaymentMethod': ['type'], 'CurrencyNote': ['ID', 'isOriginal', 'value'], 'Error': ['ID', 'type', 'message'], 'ErrorMessage': ['message'], 'TouchScreen': ['size']}
Relationships Extracted: [{'source': 'TicketDistributor', 'target': 'Ticket', 'type': 'Composition', 'description': 'The TicketDistributor issues and manages Tickets; Tickets cannot exist independently without the distributo

In [58]:
%pip install plantuml

  Using cached plantuml-0.3.0-py3-none-any.whl.metadata (2.5 kB)
Using cached plantuml-0.3.0-py3-none-any.whl (5.8 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Debugging

In [66]:
import os
import json

def generate_plantuml(classes, attributes, relationships, output_file="class_diagram.puml"):
    """
    Generates a PlantUML class diagram from identified classes, attributes, and relationships.
    """
    if not isinstance(classes, list):
        print("Error: 'classes' should be a list. Received:", type(classes))
        return
    
    plantuml_code = "@startuml\n\n"

    # Define Classes and Attributes
    for cls in classes:
        plantuml_code += f"class {cls} {{\n"
        if isinstance(attributes, dict) and cls in attributes:
            for attr in attributes[cls]:
                plantuml_code += f"  {attr}\n"
        plantuml_code += "}\n\n"

   # Define relationships
    for rel in relationships:
        source = rel["source"]
        target = rel["target"]
        rel_type = rel["type"]
        description = rel["description"]

        if rel_type == "Association":
            plantuml_code += f"{source} --> {target} : {description}\n"
        elif rel_type == "Generalization":
            plantuml_code += f"{source} --|> {target} : {description}\n"
        elif rel_type == "Aggregation":
            plantuml_code += f"{source} o-- {target} : {description}\n"
        elif rel_type == "Composition":
            plantuml_code += f"{source} *-- {target} : {description}\n"

    plantuml_code += "\n@enduml"

    # Save to file
    with open(output_file, "w") as file:
        file.write(plantuml_code)

    print(f"✅ PlantUML file '{output_file}' generated successfully.")

# 🛠 Debugging Check
print("Classes Extracted:", identified_classes)
print("Attributes Extracted:", identified_attributes)
print("Relationships Extracted:", identified_relationships)

# Generate UML
# generate_plantuml(identified_classes, identified_attributes, identified_relationships)

# To render, use: `plantuml class_diagram.puml` in the terminal


Classes Extracted: ['TicketDistributor', 'Passenger', 'Ticket', 'MetroPass', 'TimeCard', 'Transaction', 'DisplayInterface', 'PaymentMethod', 'CurrencyNote', 'Error', 'ErrorMessage', 'TouchScreen']
Attributes Extracted: {'TicketDistributor': ['ID'], 'Passenger': ['ID', 'type', 'isFrequentTraveler'], 'Ticket': ['ID', 'type', 'price', 'departureTime', 'date', 'time', 'validFrom', 'validTill', 'validityDuration', 'issuedFrom', 'destinationStation'], 'MetroPass': ['ID', 'type', 'validityDuration'], 'TimeCard': ['ID', 'type', 'validityDuration'], 'Transaction': ['ID', 'status', 'amount'], 'DisplayInterface': ['type'], 'PaymentMethod': ['type'], 'CurrencyNote': ['ID', 'isOriginal', 'value'], 'Error': ['ID', 'type', 'message'], 'ErrorMessage': ['message'], 'TouchScreen': ['size']}
Relationships Extracted: [{'source': 'TicketDistributor', 'target': 'Ticket', 'type': 'Composition', 'description': 'TicketDistributor issues and manages Tickets; Tickets cannot exist independently without the distri

In [ ]:
import json
import re

def identify_attributes(problem_statement, classes):
    """
    Identifies attributes for each class extracted from the problem statement using Gemini API.
    Ensures the output is a parsed dictionary.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to extract **attributes** for each identified class from the given problem statement.

    **Rules:**
    - Attributes describe **properties** of a class (e.g., "User" has "name", "email").
    - Ignore verbs and relationships.
    - Output should be strictly in **JSON format**, like:
      {{
          "attributes": {{
              "Class1": ["attr1", "attr2"],
              "Class2": ["attr1", "attr2"]
          }}
      }}

    **Classes Identified:**
    {classes}

    **Problem Statement:**
    {problem_statement}

    Extract attributes for each class.
    """

    model = genai.GenerativeModel("gemini-1.5-flash") 
    response = model.generate_content(prompt, generation_config={"temperature": 0})

    # Debugging: Print raw response
    response_text = response.text.strip()
    print("Raw Response from Gemini:\n", response_text)

   # Remove triple backticks and 'json' keyword
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))  # Use repr() to detect hidden characters

    # Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

     # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("attributes", {})  # Extract attributes dictionary
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

identified_attributes = identify_attributes(problem_statement, identified_classes)

print("Extracted Attributes:\n", identified_attributes)


Raw Response from Gemini:
 ```json
{
  "attributes": {
    "TicketDistributor": ["heavy components"],
    "Passenger": ["isChild", "isFrequentTraveler"],
    "Ticket": ["type", "quantity", "price", "departureTime", "date", "time", "validFrom", "validTill", "validityDuration", "ticketIssuedFrom", "destinationStation", "updatedInformation"],
    "MetroPass": [],
    "TimeCard": ["duration"], 
    "Transaction": ["type", "amount", "paymentMethod"],
    "DisplayInterface": ["type"],
    "PaymentMethod": ["type"],
    "CurrencyNote": ["isOriginal", "value"],
    "Error": [],
    "ErrorMessage": ["message"],
    "TouchScreen": []
  }
}
```
Cleaned JSON:
 '{\n  "attributes": {\n    "TicketDistributor": ["heavy components"],\n    "Passenger": ["isChild", "isFrequentTraveler"],\n    "Ticket": ["type", "quantity", "price", "departureTime", "date", "time", "validFrom", "validTill", "validityDuration", "ticketIssuedFrom", "destinationStation", "updatedInformation"],\n    "MetroPass": [],\n    "Tim

dict

In [34]:
json_input = '''{
  "attributes": {
    "TicketDistributor": ["heavy components"],
    "Passenger": ["isChild", "isFrequentTraveler"],
    "Ticket": ["type", "quantity", "price", "departureTime", "date", "time", "validFrom", "validTill", "validityDuration", "ticketIssuedFrom", "destinationStation", "updatedInformation"],
    "MetroPass": [],
    "TimeCard": ["duration"], 
    "Transaction": ["type", "amount", "paymentMethod"],
    "DisplayInterface": ["type"],
    "PaymentMethod": ["type"],
    "CurrencyNote": ["isOriginal", "value"],
    "Error": [],
    "ErrorMessage": ["message"],
    "TouchScreen": []
  }
}'''

try:
        output = json.loads(json_input)  # Directly parse JSON
        print(output.get("attributes", {})) # Extract attributes dictionary
except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        print("No output...")

{'TicketDistributor': ['heavy components'], 'Passenger': ['isChild', 'isFrequentTraveler'], 'Ticket': ['type', 'quantity', 'price', 'departureTime', 'date', 'time', 'validFrom', 'validTill', 'validityDuration', 'ticketIssuedFrom', 'destinationStation', 'updatedInformation'], 'MetroPass': [], 'TimeCard': ['duration'], 'Transaction': ['type', 'amount', 'paymentMethod'], 'DisplayInterface': ['type'], 'PaymentMethod': ['type'], 'CurrencyNote': ['isOriginal', 'value'], 'Error': [], 'ErrorMessage': ['message'], 'TouchScreen': []}


In [59]:
import re
import json
import google.generativeai as genai

def identify_relationships(problem_statement, classes, attributes):
    """
    Identifies relationships between extracted classes using Gemini API with CoT prompting.
    Extracts Association, Generalization (Inheritance), Aggregation, and Composition.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **all types of relationships** between the given classes based on the problem statement.

    **Types of Relationships:**
    1️⃣ **Association** - One class interacts with another. Example: "User borrows Book".
    2️⃣ **Generalization (Inheritance)** - One class is a specialized form of another. Example: "Admin is a subclass of User".
    3️⃣ **Aggregation** - One class is made up of another, but they have independent lifecycles. Example: "Library has Books, but Books exist independently".
    4️⃣ **Composition** - A stronger form of Aggregation, where the part **cannot exist** without the whole. Example: "House has Rooms, Rooms cannot exist without a House".

    **Step-by-Step Process:**
    - Identify the **verbs** and **context clues** in the problem statement to find relationships.
    - Classify the relationship into one of the four types above.
    - Provide the relationships in **strict JSON format**, like this:
      ```json
      {{
          "relationships": [
              {{"source": "ClassA", "target": "ClassB", "type": "Generalization", "description": "ClassA is a subclass of ClassB"}},
              {{"source": "ClassC", "target": "ClassD", "type": "Aggregation", "description": "ClassC contains ClassD but ClassD can exist independently"}},
              {{"source": "ClassE", "target": "ClassF", "type": "Composition", "description": "ClassE owns ClassF and ClassF cannot exist without ClassE"}}
          ]
      }}
      ```

    **Classes Identified:**
    {classes}

    **Attributes Identified:**
    {attributes}

    **Problem Statement:**
    {problem_statement}

    Now extract **all types of relationships** between the classes.
    """

    model = genai.GenerativeModel("gemini-1.5-flash") 
    response = model.generate_content(prompt, generation_config={"temperature": 0, "top_p": 1, "top_k": 1})

    # Debugging: Print raw response
    response_text = response.text.strip()

    # Remove triple backticks and 'json' keyword
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Debugging: Print cleaned JSON before parsing
    print("🛠 Cleaned JSON:\n", repr(cleaned_text))

    # Check if cleaned_text is empty
    if not cleaned_text:
        print("❌ Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("relationships", [])  
    except json.JSONDecodeError as e:
        print("❌ JSON parsing error:", str(e))
        return {}

# Call function and print results
identified_relationships = identify_relationships(problem_statement, identified_classes, identified_attributes)

print("✅ Relationships Identified:", identified_relationships)


🛠 Cleaned JSON:
 '{\n  "relationships": [\n    {\n      "source": "TicketDistributor",\n      "target": "Ticket",\n      "type": "Composition",\n      "description": "TicketDistributor issues Tickets, and Tickets cannot exist independently without being issued."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "MetroPass",\n      "type": "Association",\n      "description": "TicketDistributor reads MetroPass information."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "TimeCard",\n      "type": "Association",\n      "description": "TicketDistributor reads TimeCard information."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "Transaction",\n      "type": "Composition",\n      "description": "TicketDistributor creates and manages Transactions, which cannot exist independently."\n    },\n    {\n      "source": "TicketDistributor",\n      "target": "DisplayInterface",\n      "type": "Composition",\n      "description": "Tic